# [Lab 01] Indice invertido

In [1]:
import pandas as pd
import nltk

In [2]:
dados = pd.read_csv("noticias_estadao.csv")

Cria o indice invertido a partir do noticias_estadao.csv

In [3]:
dados["noticia"] = dados.titulo +" "+ dados.conteudo #cria nova coluna

mapa_tokens = {}
indice_invertido = {}

#cria um mapa relacionando o indice da noticia e suas palavras
for i in range(dados.noticia.count()):
    tokens = set(nltk.word_tokenize(dados.noticia[i]))
    mapa_tokens[dados.idNoticia[i]] = tokens

#cria o indice invertido    
for id_noticia, tokens in mapa_tokens.items():
    for token in tokens:
        token = token.lower()
        if token in indice_invertido:
            indice_invertido.get(token).add(id_noticia)
        else:
            indice_invertido[token] = set([id_noticia])


Cria as funções que serão utilizadas para as buscas Booleanas

In [4]:
def func_or(word1, word2):
    return set(indice_invertido[word1] + indice_invertido[word2])
    
    
def func_and(word1,word2):
    return set(indice_invertido[word1]) and set(indice_invertido[word2])

def and_conjuntivo(lista_palavras):
    
    ids_por_palavra = [indice_invertido.get(palavra) for palavra in lista_palavras]
    
    if None in ids_por_palavra:
        return []
    else:
        resultado = set.intersection(*ids_por_palavra)
        return list(resultado)
    

def search(entrada):
    
    # separa a entrada em termos para as operacoes de OR
    termos_do_or = entrada.split("OR")
    termos_do_or = [termo.strip() for termo in termos_do_or]
    
    #resolve os ANDs de cada termo do OR
    resultado_dos_ands = []
    for elemento in termos_do_or:
        termos_do_and = elemento.split("AND")
        termos_do_and = [termo.strip().lower() for termo in termos_do_and]
        resultado_dos_ands.append(and_conjuntivo(termos_do_and))
        
    #resolve os ORs
    result = set().union(*resultado_dos_ands)
    return list(result)
    


# Testes

1- debate, presidenciável (AND e OR)

In [5]:
assert len(search("debate OR presidencial")) == 1770
assert len(search("debate AND presidencial")) == 201

2- presidenciáveis, corruptos (AND e OR)

In [6]:
assert len(search("presidenciáveis OR corruptos")) == 164
assert len(search("presidenciáveis AND corruptos")) == 0

3- Belo, Horizonte (AND e OR)

In [7]:
assert len(search("Belo OR Horizonte")) == 331
assert len(search("Belo AND Horizonte")) == 242

4- Campina, Grande, Paraíba

In [8]:
assert len(search("Campina AND Grande AND Paraíba")) == 8